In [1]:
from transformers import GPT2TokenizerFast, ViTImageProcessor, VisionEncoderDecoderModel
from torch.utils.data import Dataset
from torchtext.data import get_tokenizer
import requests
import torch
import numpy as np
from PIL import Image
import pickle
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
model_raw = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

config.json:   0%|          | 0.00/4.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/982M [00:00<?, ?B/s]

In [3]:
image_processor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer       = GPT2TokenizerFast.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

In [4]:
def show_n_generate(url, greedy = True, model = model_raw, num_captions = 4):
    image = Image.open(requests.get(url, stream =True).raw)
    pixel_values   = image_processor(image, return_tensors ="pt").pixel_values

    captions = []
    for _ in range(num_captions):
        generated_ids = model.generate(
            pixel_values,
            do_sample=True,
            max_new_tokens=30,
            top_k=5
        )
        generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        captions.append(generated_text)

    return captions

## Store All 250 Images in this list

In [5]:
url_list = ["https://i.postimg.cc/FRTTKfLK/self-dataset-img750.jpg", "https://i.postimg.cc/Hsy9qvBn/self-dataset-img754.jpg",
            "https://i.postimg.cc/g0FMnFtf/self-dataset-img751.jpg", "https://i.postimg.cc/zfCxht93/self-dataset-img752.jpg",
            "https://i.postimg.cc/43cdZMmL/self-dataset-img753.jpg", "https://i.postimg.cc/4Ny201hF/self-dataset-img755.jpg",
            "https://i.postimg.cc/X7P16yfN/self-dataset-img756.jpg", "https://i.postimg.cc/bwG0xC2p/self-dataset-img757.jpg",
            "https://i.postimg.cc/YSKF1pHz/self-dataset-img758.jpg", "https://i.postimg.cc/jSJgxdcH/self-dataset-img759.jpg",
            "https://i.postimg.cc/D0M6htbp/self-dataset-img760.jpg", "https://i.postimg.cc/dQ4SnfDy/self-dataset-img761.jpg",
            "https://i.postimg.cc/8CZrxLvt/self-dataset-img762.jpg", "https://i.postimg.cc/G3Gkz7Kj/self-dataset-img763.jpg",
            "https://i.postimg.cc/hvjsHSh7/self-dataset-img764.jpg", "https://i.postimg.cc/1Xn6G4fM/self-dataset-img765.jpg",
            "https://i.postimg.cc/L5cPNRPS/self-dataset-img766.jpg", "https://i.postimg.cc/zfrjqwVN/self-dataset-img767.jpg",
            "https://i.postimg.cc/L8MfP9RB/self-dataset-img768.jpg", "https://i.postimg.cc/Hj1JbzGs/self-dataset-img769.jpg",
            "https://i.postimg.cc/tgqxrBss/self-dataset-img770.jpg", "https://i.postimg.cc/GmJDJWm0/self-dataset-img771.jpg",
            "https://i.postimg.cc/KYpNS13r/self-dataset-img772.jpg", "https://i.postimg.cc/cCngKjbb/self-dataset-img773.jpg",
            "https://i.postimg.cc/nzM3C4fP/self-dataset-img774.jpg", "https://i.postimg.cc/wjcfPxk0/self-dataset-img775.jpg",
            "https://i.postimg.cc/zDVQBrJG/self-dataset-img776.jpg", "https://i.postimg.cc/8zH44K1Y/self-dataset-img777.jpg",
            "https://i.postimg.cc/CKhPVyPY/self-dataset-img778.jpg", "https://i.postimg.cc/0yxZpRqm/self-dataset-img779.jpg",
            "https://i.postimg.cc/mZzd4ywc/self-dataset-img780.jpg", "https://i.postimg.cc/7YSmGPjw/self-dataset-img781.jpg",
            "https://i.postimg.cc/Dz0rg7pW/self-dataset-img782.jpg", "https://i.postimg.cc/fW9vpTzm/self-dataset-img783.jpg",
            "https://i.postimg.cc/5yLqCp7n/self-dataset-img784.jpg", "https://i.postimg.cc/3JjFrSDw/self-dataset-img785.jpg",
            "https://i.postimg.cc/CLp4gT5t/self-dataset-img786.jpg", "https://i.postimg.cc/J4QjCs9D/self-dataset-img787.jpg",
            "https://i.postimg.cc/QxDczQnZ/self-dataset-img788.jpg", "https://i.postimg.cc/sxmxqhv3/self-dataset-img789.jpg",
            "https://i.postimg.cc/wMXLvMMr/self-dataset-img790.jpg", "https://i.postimg.cc/wj05L128/self-dataset-img791.jpg", "https://i.postimg.cc/k4MDBbFL/self-dataset-img792.jpg",
            "https://i.postimg.cc/9Qr0gzVj/self-dataset-img793.jpg", "https://i.postimg.cc/wTFRbSmD/self-dataset-img794.jpg",
            "https://i.postimg.cc/qRfN3rPH/self-dataset-img795.jpg", "https://i.postimg.cc/5ySQJrkd/self-dataset-img796.jpg",
            "https://i.postimg.cc/L43JnrvG/self-dataset-img797.jpg", "https://i.postimg.cc/W13qRXGC/self-dataset-img798.jpg",
            "https://i.postimg.cc/7PKxvDKP/self-dataset-img799.jpg", "https://i.postimg.cc/RFcN18m6/self-dataset-img800.jpg",
            "https://i.postimg.cc/jd8ChPpX/self-dataset-img801.jpg", "https://i.postimg.cc/yx1Ym8nt/self-dataset-img802.jpg",
            "https://i.postimg.cc/XqG1Dw2C/self-dataset-img803.jpg", "https://i.postimg.cc/SsJpkYTy/self-dataset-img804.jpg",
            "https://i.postimg.cc/mkmZ8W5D/self-dataset-img805.jpg", "https://i.postimg.cc/HxCLL2Mp/self-dataset-img806.jpg",
            "https://i.postimg.cc/8cjxg8Cs/self-dataset-img807.jpg", "https://i.postimg.cc/rwBVhJQS/self-dataset-img808.jpg",
            "https://i.postimg.cc/FKp6fFd7/self-dataset-img809.jpg", "https://i.postimg.cc/wjyxcPcq/self-dataset-img810.jpg",
            "https://i.postimg.cc/Hx2TkCH2/self-dataset-img811.jpg", "https://i.postimg.cc/VsV3BN7t/self-dataset-img812.jpg",
            "https://i.postimg.cc/654FHNSm/self-dataset-img813.jpg", "https://i.postimg.cc/rs3pnYXL/self-dataset-img814.jpg","https://i.postimg.cc/SNWbShrz/self-dataset-img815.jpg",
            "https://i.postimg.cc/906g3rSN/self-dataset-img816.jpg", "https://i.postimg.cc/vZtyzsxd/self-dataset-img817.jpg",
            "https://i.postimg.cc/RZMyvnhj/self-dataset-img818.jpg", "https://i.postimg.cc/jqnr1Nnc/self-dataset-img819.jpg",
            "https://i.postimg.cc/8cM9zRhV/self-dataset-img820.jpg", "https://i.postimg.cc/JzLmZnnq/self-dataset-img821.jpg",
            "https://i.postimg.cc/0QF1Q5F7/self-dataset-img822.jpg", "https://i.postimg.cc/XvYRky0w/self-dataset-img823.jpg",
            "https://i.postimg.cc/ydw2KQ3P/self-dataset-img824.jpg", "https://i.postimg.cc/251H5dDX/self-dataset-img825.jpg",
            "https://i.postimg.cc/V6S23Qvv/self-dataset-img826.jpg", "https://i.postimg.cc/LsQrC6HG/self-dataset-img827.jpg",
            "https://i.postimg.cc/L41rDpG0/self-dataset-img828.jpg", "https://i.postimg.cc/5NWDkrV9/self-dataset-img829.jpg",
            "https://i.postimg.cc/5N5Dcm4B/self-dataset-img830.jpg", "https://i.postimg.cc/jqPFNhJp/self-dataset-img831.jpg",
            "https://i.postimg.cc/nc1SSc2T/self-dataset-img832.jpg", "https://i.postimg.cc/5NJkmFB6/self-dataset-img833.jpg",
            "https://i.postimg.cc/t4SSySLx/self-dataset-img834.jpg", "https://i.postimg.cc/vBdWd5Hn/self-dataset-img835.jpg",
            "https://i.postimg.cc/vBqHvdjm/self-dataset-img836.jpg", "https://i.postimg.cc/vBLSRYMQ/self-dataset-img837.jpg",
            "https://i.postimg.cc/TwphFwnb/self-dataset-img838.jpg", "https://i.postimg.cc/vZgqYLM2/self-dataset-img839.jpg",
            "https://i.postimg.cc/pdQBrsSC/self-dataset-img840.jpg", "https://i.postimg.cc/5tPS8XxQ/self-dataset-img841.jpg",
            "https://i.postimg.cc/DyYp3YVb/self-dataset-img842.jpg", "https://i.postimg.cc/4dTFFKDL/self-dataset-img843.jpg",
            "https://i.postimg.cc/13fTg10H/self-dataset-img844.jpg", "https://i.postimg.cc/15TBjn6G/self-dataset-img845.jpg",
            "https://i.postimg.cc/dt0NXcfY/self-dataset-img846.jpg", "https://i.postimg.cc/RZHd5SQj/self-dataset-img847.jpg",
            "https://i.postimg.cc/50xSCwdV/self-dataset-img848.jpg", "https://i.postimg.cc/NfLbD7yG/self-dataset-img849.jpg",
            "https://i.postimg.cc/BvfgJrmt/self-dataset-img850.jpg", "https://i.postimg.cc/kMTfWhgh/self-dataset-img851.jpg",
            "https://i.postimg.cc/tRvNsZBK/self-dataset-img852.jpg", "https://i.postimg.cc/D01Gp0Kg/self-dataset-img853.jpg",
            "https://i.postimg.cc/G22sRznH/self-dataset-img854.jpg", "https://i.postimg.cc/nhp454rQ/self-dataset-img855.jpg",
            "https://i.postimg.cc/KzJrrRxg/self-dataset-img856.jpg", "https://i.postimg.cc/PJ06YPQQ/self-dataset-img857.jpg",
            "https://i.postimg.cc/dQzP811f/self-dataset-img858.jpg", "https://i.postimg.cc/Tw4QvpFv/self-dataset-img859.jpg",
            "https://i.postimg.cc/kXxNf1Jm/self-dataset-img860.jpg", "https://i.postimg.cc/XJCBXqXb/self-dataset-img861.jpg",
            "https://i.postimg.cc/44cpxLh9/self-dataset-img862.jpg", "https://i.postimg.cc/7PCrG7x7/self-dataset-img863.jpg",
            "https://i.postimg.cc/prmGmDzd/self-dataset-img864.jpg", "https://i.postimg.cc/6qV7LPdb/self-dataset-img865.jpg",
            "https://i.postimg.cc/5NHTyMSm/self-dataset-img866.jpg", "https://i.postimg.cc/XvgDmNnn/self-dataset-img867.jpg",
            "https://i.postimg.cc/htDCxW9w/self-dataset-img868.jpg", "https://i.postimg.cc/ZYLpN9Vb/self-dataset-img869.jpg",
            "https://i.postimg.cc/7Z52jKDh/self-dataset-img870.jpg", "https://i.postimg.cc/3wbbYS0q/self-dataset-img871.jpg",
            "https://i.postimg.cc/rws1M8G8/self-dataset-img872.jpg", "https://i.postimg.cc/j2YcSGxM/self-dataset-img873.jpg",
            "https://i.postimg.cc/Qxx5JXyp/self-dataset-img874.jpg", "https://i.postimg.cc/qRsnjgrG/self-dataset-img875.jpg",
            "https://i.postimg.cc/3JnXsyvS/self-dataset-img876.jpg", "https://i.postimg.cc/6TskwHgK/self-dataset-img877.jpg",
            "https://i.postimg.cc/XYtCW978/self-dataset-img878.jpg", "https://i.postimg.cc/Gp1mzKmZ/self-dataset-img879.jpg",
            "https://i.postimg.cc/zvHssmdp/self-dataset-img880.jpg", "https://i.postimg.cc/nzrb19By/self-dataset-img881.jpg",
            "https://i.postimg.cc/jjQd1fWC/self-dataset-img882.jpg", "https://i.postimg.cc/ZRsZkxJY/self-dataset-img883.jpg",
            "https://i.postimg.cc/ZR5ZDMtD/self-dataset-img884.jpg", "https://i.postimg.cc/htmG96cL/self-dataset-img885.jpg",
            "https://i.postimg.cc/Y047tLS6/self-dataset-img886.jpg", "https://i.postimg.cc/44MZBP8B/self-dataset-img887.jpg",
            "https://i.postimg.cc/NMB7hRpn/self-dataset-img888.jpg", "https://i.postimg.cc/NGTYCTWt/self-dataset-img889.jpg",
            "https://i.postimg.cc/VkZ3kL34/self-dataset-img890.jpg", "https://i.postimg.cc/sg3VhXHr/self-dataset-img891.jpg",
            "https://i.postimg.cc/tC2cB4Cy/self-dataset-img892.jpg", "https://i.postimg.cc/WbLpsSMt/self-dataset-img893.jpg",
            "https://i.postimg.cc/brLmVmPL/self-dataset-img894.jpg", "https://i.postimg.cc/W31F689s/self-dataset-img895.jpg",
            "https://i.postimg.cc/1XyWKLkf/self-dataset-img896.jpg", "https://i.postimg.cc/rySXNTrF/self-dataset-img897.jpg",
            "https://i.postimg.cc/43tBRfZY/self-dataset-img898.jpg", "https://i.postimg.cc/zGvc6YVh/self-dataset-img899.jpg",
            "https://i.postimg.cc/655j4xVP/self-dataset-img900.jpg", "https://i.postimg.cc/wjn0ryKK/self-dataset-img901.jpg",
            "https://i.postimg.cc/FsBZfdHf/self-dataset-img902.jpg", "https://i.postimg.cc/Y288Hycz/self-dataset-img903.jpg",
            "https://i.postimg.cc/MHZDB1BB/self-dataset-img904.jpg", "https://i.postimg.cc/m2Dj2NBf/self-dataset-img905.jpg",
            "https://i.postimg.cc/2SLxnk10/self-dataset-img906.jpg", "https://i.postimg.cc/hG9b32bx/self-dataset-img907.jpg",
            "https://i.postimg.cc/tTkd97c4/self-dataset-img908.jpg", "https://i.postimg.cc/Xv3yX4dG/self-dataset-img909.jpg",
            "https://i.postimg.cc/28sWR45x/self-dataset-img910.jpg", "https://i.postimg.cc/J09s8LRh/self-dataset-img911.jpg",
            "https://i.postimg.cc/HnLwndG4/self-dataset-img912.jpg", "https://i.postimg.cc/L40fvdKw/self-dataset-img913.jpg",
            "https://i.postimg.cc/SQZL528Z/self-dataset-img914.jpg", "https://i.postimg.cc/DyJGMC7Z/self-dataset-img915.jpg",
            "https://i.postimg.cc/bdcwM9RD/self-dataset-img916.jpg", "https://i.postimg.cc/ZKd9cCLv/self-dataset-img917.jpg",
            "https://i.postimg.cc/y65gcF0z/self-dataset-img918.jpg", "https://i.postimg.cc/nrtLS12s/self-dataset-img919.jpg",
            "https://i.postimg.cc/XqyvQz96/self-dataset-img920.jpg", "https://i.postimg.cc/76R61kGN/self-dataset-img921.jpg",
            "https://i.postimg.cc/Pq2NxZd9/self-dataset-img922.jpg", "https://i.postimg.cc/g0JL34tm/self-dataset-img923.jpg",
            "https://i.postimg.cc/xTDmBv83/self-dataset-img924.jpg", "https://i.postimg.cc/1z9fsM1D/self-dataset-img925.jpg",
            "https://i.postimg.cc/fWjWTm5K/self-dataset-img926.jpg", "https://i.postimg.cc/Qtv785bs/self-dataset-img927.jpg",
            "https://i.postimg.cc/KYHkfLCL/self-dataset-img928.jpg", "https://i.postimg.cc/rmDW2g5Q/self-dataset-img929.jpg",
            "https://i.postimg.cc/2SxnQ1zF/self-dataset-img930.jpg", "https://i.postimg.cc/L5f1nXCP/self-dataset-img931.jpg",
            "https://i.postimg.cc/sXGBNxRL/self-dataset-img932.jpg", "https://i.postimg.cc/SN694nzF/self-dataset-img933.jpg",
            "https://i.postimg.cc/K8mM8mnK/self-dataset-img934.jpg", "https://i.postimg.cc/c4SKFdkD/self-dataset-img935.jpg",
            "https://i.postimg.cc/zDWbwy7b/self-dataset-img936.jpg", "https://i.postimg.cc/9QzcpS28/self-dataset-img937.jpg",
            "https://i.postimg.cc/d3M7PFsV/self-dataset-img938.jpg", "https://i.postimg.cc/mgZPG8sC/self-dataset-img939.jpg",
            "https://i.postimg.cc/mrCfpW5w/self-dataset-img940.jpg", "https://i.postimg.cc/MHxWmpZV/self-dataset-img941.jpg",
            "https://i.postimg.cc/NMXMxn6q/self-dataset-img942.jpg", "https://i.postimg.cc/SNR0FMPj/self-dataset-img943.jpg",
            "https://i.postimg.cc/C1729hHj/self-dataset-img944.jpg", "https://i.postimg.cc/7YGBnMc1/self-dataset-img945.jpg",
            "https://i.postimg.cc/6p4DTNtX/self-dataset-img946.jpg", "https://i.postimg.cc/02yvk3zk/self-dataset-img947.jpg",
            "https://i.postimg.cc/G2j0rJrT/self-dataset-img948.jpg", "https://i.postimg.cc/dV7KdyRg/self-dataset-img949.jpg",
            "https://i.postimg.cc/Fs76nQfL/self-dataset-img950.jpg", "https://i.postimg.cc/qR4YmhnJ/self-dataset-img951.jpg",
            "https://i.postimg.cc/0N8dFptB/self-dataset-img952.jpg", "https://i.postimg.cc/4dfLvCkn/self-dataset-img953.jpg",
            "https://i.postimg.cc/3r61N0Rk/self-dataset-img954.jpg", "https://i.postimg.cc/tTv5hYNm/self-dataset-img955.jpg",
            "https://i.postimg.cc/x1LtWj9c/self-dataset-img956.jpg", "https://i.postimg.cc/C5dctjKg/self-dataset-img957.jpg",
            "https://i.postimg.cc/Hk1t4GbH/self-dataset-img958.jpg", "https://i.postimg.cc/YS38CXG7/self-dataset-img959.jpg",
            "https://i.postimg.cc/gkZDZcXN/self-dataset-img960.jpg", "https://i.postimg.cc/kgNvTjyS/self-dataset-img961.jpg",
            "https://i.postimg.cc/2S5nxz3s/self-dataset-img962.jpg", "https://i.postimg.cc/Qx1zrJb8/self-dataset-img963.jpg",
            "https://i.postimg.cc/JzC9y2nH/self-dataset-img964.jpg", "https://i.postimg.cc/W3yGdxsB/self-dataset-img965.jpg",
            "https://i.postimg.cc/SRzWCVsJ/self-dataset-img966.jpg", "https://i.postimg.cc/BvMc0XkQ/self-dataset-img967.jpg",
            "https://i.postimg.cc/PJ8RJWnM/self-dataset-img968.jpg", "https://i.postimg.cc/65tryVK5/self-dataset-img969.jpg",
            "https://i.postimg.cc/P5GkWq6x/self-dataset-img970.jpg", "https://i.postimg.cc/1X5zZzxP/self-dataset-img971.jpg",
            "https://i.postimg.cc/FsLMH49Q/self-dataset-img972.jpg", "https://i.postimg.cc/63QsW4QH/self-dataset-img973.jpg",
            "https://i.postimg.cc/FRzq5tP8/self-dataset-img974.jpg", "https://i.postimg.cc/3W5bvkKQ/self-dataset-img975.jpg",
            "https://i.postimg.cc/gJ77GMQJ/self-dataset-img976.jpg", "https://i.postimg.cc/hvFH9yrY/self-dataset-img977.jpg",
            "https://i.postimg.cc/DzphbPRV/self-dataset-img978.jpg", "https://i.postimg.cc/zDsJjpv8/self-dataset-img979.jpg",
            "https://i.postimg.cc/mZH4tMwq/self-dataset-img980.jpg", "https://i.postimg.cc/nhVNYQ4n/self-dataset-img981.jpg",
            "https://i.postimg.cc/59LcxJ9W/self-dataset-img982.jpg", "https://i.postimg.cc/ZYFf75Kn/self-dataset-img983.jpg", "https://i.postimg.cc/bYBdwdGF/self-dataset-img984.jpg",
            "https://i.postimg.cc/brkfWcr8/self-dataset-img985.jpg", "https://i.postimg.cc/28dfxDg7/self-dataset-img986.jpg",
            "https://i.postimg.cc/3JFPvZDL/self-dataset-img987.jpg", "https://i.postimg.cc/rpsXctsf/self-dataset-img988.jpg",
            "https://i.postimg.cc/y8v4wT1J/self-dataset-img989.jpg", "https://i.postimg.cc/Gtk6zFgm/self-dataset-img990.jpg",
            "https://i.postimg.cc/bvsM4cVM/self-dataset-img991.jpg", "https://i.postimg.cc/66dPkhz9/self-dataset-img992.jpg",
            "https://i.postimg.cc/6qwMWDzB/self-dataset-img993.jpg", "https://i.postimg.cc/dtwfwTSG/self-dataset-img994.jpg",
            "https://i.postimg.cc/PrSSHVDk/self-dataset-img995.jpg", "https://i.postimg.cc/d0pXYHz0/self-dataset-img996.jpg",
            "https://i.postimg.cc/9QfNqH2M/self-dataset-img997.jpg", "https://i.postimg.cc/2S32SddL/self-dataset-img998.jpg",
            "https://i.postimg.cc/L5TNnN0L/self-dataset-img999.jpg"]


len(url_list)

250

## First 50 Images

In [6]:
# Create an empty DataFrame to store the results
columns = ["URL", "Caption"]
df1 = pd.DataFrame(columns=columns)

# Create a list of dictionaries to store the results
data = []

# Iterate through each URL
for url in url_list[ : 50]:
    # Generate 10 captions for each image
    captions = show_n_generate(url, model=model_raw, num_captions=4, greedy = True)
    
    # Append the results to the list
    for caption in captions:
        data.append({"URL": url, "Caption": caption})

# Create the DataFrame
df1 = pd.DataFrame(data)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


In [7]:
df1

,URL,Caption
0,https://i.postimg.cc/FRTTKfLK/self-dataset-img...,a crowded beach with people on the beach
1,https://i.postimg.cc/FRTTKfLK/self-dataset-img...,a large group of people laying in the sand
2,https://i.postimg.cc/FRTTKfLK/self-dataset-img...,a large group of people are laying out on the ...
3,https://i.postimg.cc/FRTTKfLK/self-dataset-img...,people are laying out and playing with their s...
4,https://i.postimg.cc/Hsy9qvBn/self-dataset-img...,a man is standing with a paddle in a river
...,...,...
195,https://i.postimg.cc/W13qRXGC/self-dataset-img...,a cut up meat and some vegetables on a cutting...
196,https://i.postimg.cc/7PKxvDKP/self-dataset-img...,a dog sitting on a carpet
197,https://i.postimg.cc/7PKxvDKP/self-dataset-img...,a white dog sitting on a white rug
198,https://i.postimg.cc/7PKxvDKP/self-dataset-img...,a dog with long hair laying on a blanket


## Second 50 Images

In [8]:
# Create an empty DataFrame to store the results
columns = ["URL", "Caption"]
df2 = pd.DataFrame(columns=columns)

# Create a list of dictionaries to store the results
data = []

# Iterate through each URL
for url in url_list[50: 100]:
    # Generate 10 captions for each image
    captions = show_n_generate(url, model=model_raw, num_captions=4, greedy = True)
    
    # Append the results to the list
    for caption in captions:
        data.append({"URL": url, "Caption": caption})

# Create the DataFrame
df2 = pd.DataFrame(data)

In [9]:
df2

,URL,Caption
0,https://i.postimg.cc/RFcN18m6/self-dataset-img...,a batter swinging at a ball in a baseball game
1,https://i.postimg.cc/RFcN18m6/self-dataset-img...,the catcher and umpire are looking at a ball
2,https://i.postimg.cc/RFcN18m6/self-dataset-img...,a baseball player swinging a baseball bat at a...
3,https://i.postimg.cc/RFcN18m6/self-dataset-img...,a batter and a catcher are playing baseball
4,https://i.postimg.cc/jd8ChPpX/self-dataset-img...,the brown horse is standing in a grassy field
...,...,...
195,https://i.postimg.cc/50xSCwdV/self-dataset-img...,a person riding a wave on a surf board
196,https://i.postimg.cc/NfLbD7yG/self-dataset-img...,an older man is eating pizza in front of an ol...
197,https://i.postimg.cc/NfLbD7yG/self-dataset-img...,a man eating a slice of pizza in a restaurant
198,https://i.postimg.cc/NfLbD7yG/self-dataset-img...,a man eating a piece of bread on a tray with a...


## Third 50 Images

In [10]:
# Create an empty DataFrame to store the results
columns = ["URL", "Caption"]
df3 = pd.DataFrame(columns=columns)

# Create a list of dictionaries to store the results
data = []

# Iterate through each URL
for url in url_list[100: 150]:
    # Generate 10 captions for each image
    captions = show_n_generate(url, model=model_raw, num_captions=4, greedy = True)
    
    # Append the results to the list
    for caption in captions:
        data.append({"URL": url, "Caption": caption})

# Create the DataFrame
df3 = pd.DataFrame(data)

In [11]:
df3

,URL,Caption
0,https://i.postimg.cc/BvfgJrmt/self-dataset-img...,a woman holding a basket of fruit in front of ...
1,https://i.postimg.cc/BvfgJrmt/self-dataset-img...,the woman is putting fruit into the bowl
2,https://i.postimg.cc/BvfgJrmt/self-dataset-img...,a man holding two baskets filled with bread
3,https://i.postimg.cc/BvfgJrmt/self-dataset-img...,a man holding a basket full of food on a table
4,https://i.postimg.cc/kMTfWhgh/self-dataset-img...,a man holding a yellow fish on a wooden board
...,...,...
195,https://i.postimg.cc/43tBRfZY/self-dataset-img...,"an ornate, ornate bathroom with a clock on the..."
196,https://i.postimg.cc/zGvc6YVh/self-dataset-img...,a young child in a black suit and tie is in an...
197,https://i.postimg.cc/zGvc6YVh/self-dataset-img...,a young child is working on a piece of furniture
198,https://i.postimg.cc/zGvc6YVh/self-dataset-img...,a young woman in a black shirt and a black chair


## Fourth 50 Images

In [12]:
# Create an empty DataFrame to store the results
columns = ["URL", "Caption"]
df4 = pd.DataFrame(columns=columns)

# Create a list of dictionaries to store the results
data = []

# Iterate through each URL
for url in url_list[150: 200]:
    # Generate 4 captions for each image
    captions = show_n_generate(url, model=model_raw, num_captions=4, greedy = True)
    
    # Append the results to the list
    for caption in captions:
        data.append({"URL": url, "Caption": caption})

# Create the DataFrame
df4 = pd.DataFrame(data)

In [13]:
df4

,URL,Caption
0,https://i.postimg.cc/655j4xVP/self-dataset-img...,two women are sitting at a table with orange j...
1,https://i.postimg.cc/655j4xVP/self-dataset-img...,the woman is holding a glass in front of the o...
2,https://i.postimg.cc/655j4xVP/self-dataset-img...,a woman sitting down with a glass and a plate ...
3,https://i.postimg.cc/655j4xVP/self-dataset-img...,the woman has a glass of juice and a banana in...
4,https://i.postimg.cc/wjn0ryKK/self-dataset-img...,three people sitting at a table having a meal
...,...,...
195,https://i.postimg.cc/G2j0rJrT/self-dataset-img...,a man riding on top of a surfboard on a wave
196,https://i.postimg.cc/dV7KdyRg/self-dataset-img...,a large group of people in a sailboat
197,https://i.postimg.cc/dV7KdyRg/self-dataset-img...,a sailboat with people riding on it
198,https://i.postimg.cc/dV7KdyRg/self-dataset-img...,a sail boat on the water with people standing ...


## Fifth 50 Images

In [14]:
# Create an empty DataFrame to store the results
columns = ["URL", "Caption"]
df5 = pd.DataFrame(columns=columns)

# Create a list of dictionaries to store the results
data = []

# Iterate through each URL
for url in url_list[200: ]:
    # Generate 10 captions for each image
    captions = show_n_generate(url, model=model_raw, num_captions=4, greedy = True)
    
    # Append the results to the list
    for caption in captions:
        data.append({"URL": url, "Caption": caption})

# Create the DataFrame
df5 = pd.DataFrame(data)

In [15]:
df5

,URL,Caption
0,https://i.postimg.cc/Fs76nQfL/self-dataset-img...,a surfer is riding a large wave on a board
1,https://i.postimg.cc/Fs76nQfL/self-dataset-img...,a surfer rides the waves on his board
2,https://i.postimg.cc/Fs76nQfL/self-dataset-img...,a man surfing on a wave with a surfboard
3,https://i.postimg.cc/Fs76nQfL/self-dataset-img...,a person on a surfboard on a wave in the ocean
4,https://i.postimg.cc/qR4YmhnJ/self-dataset-img...,a surfer riding a wave on a surfboard
...,...,...
195,https://i.postimg.cc/2S32SddL/self-dataset-img...,the worker is fixing a pipe on the building
196,https://i.postimg.cc/L5TNnN0L/self-dataset-img...,a book is sitting on a desk next to a notebook
197,https://i.postimg.cc/L5TNnN0L/self-dataset-img...,a book is sitting on a book holder next to a pen
198,https://i.postimg.cc/L5TNnN0L/self-dataset-img...,a book sitting on top of a table with an orang...


## Combining All Datasets and Saving
* **NAME THE FILE WITH YOUR NAME**
* **Example:** name_startimg_to_endimg (Mubashir_0_to_249)

In [16]:
final_df = pd.concat([df1, df2, df3, df4, df5], axis = 0, ignore_index = True)

In [17]:
final_df

,URL,Caption
0,https://i.postimg.cc/FRTTKfLK/self-dataset-img...,a crowded beach with people on the beach
1,https://i.postimg.cc/FRTTKfLK/self-dataset-img...,a large group of people laying in the sand
2,https://i.postimg.cc/FRTTKfLK/self-dataset-img...,a large group of people are laying out on the ...
3,https://i.postimg.cc/FRTTKfLK/self-dataset-img...,people are laying out and playing with their s...
4,https://i.postimg.cc/Hsy9qvBn/self-dataset-img...,a man is standing with a paddle in a river
...,...,...
995,https://i.postimg.cc/2S32SddL/self-dataset-img...,the worker is fixing a pipe on the building
996,https://i.postimg.cc/L5TNnN0L/self-dataset-img...,a book is sitting on a desk next to a notebook
997,https://i.postimg.cc/L5TNnN0L/self-dataset-img...,a book is sitting on a book holder next to a pen
998,https://i.postimg.cc/L5TNnN0L/self-dataset-img...,a book sitting on top of a table with an orang...


In [18]:
final_df.to_csv("sunyah_750_to_999.csv", index = False)